# 📤 Módulo 01c - Carga a SQL Server

Carga las **27 tablas** generadas por 01a y 01b a SQL Server.

In [1]:
import pickle
import pyodbc
from sqlalchemy import create_engine
import urllib.parse
from pathlib import Path

CACHE_DIR = Path('../../outputs/pickle_cache')
print('📂 Cargando DataFrames desde pickle...')


📂 Cargando DataFrames desde pickle...


### 📥 Cargar DataFrames INE y Eurostat

In [2]:
# Cargar todos los pickles
def cargar_pickle(nombre):
    ruta = CACHE_DIR / f'{nombre}.pkl'
    with open(ruta, 'rb') as f:
        return pickle.load(f)

# DataFrames INE
df_ipc_anual = cargar_pickle('df_ipc_anual')
df_umbral_limpio = cargar_pickle('df_umbral_limpio')
df_carencia_limpio = cargar_pickle('df_carencia_material')
df_arope_edad_sexo_limpio = cargar_pickle('df_arope_edad_sexo')
df_arope_hogar_limpio = cargar_pickle('df_arope_hogar')
df_arope_laboral_limpio = cargar_pickle('df_arope_laboral')
df_gini_ccaa_limpio = cargar_pickle('df_gini_ccaa')
df_renta_decil_limpio = cargar_pickle('df_renta_decil')
df_pob_edad_sexo_nacion_limpio = cargar_pickle('df_poblacion')
df_pob_edad_sexo_ccaa_limpio = cargar_pickle('df_poblacion_ccaa_edad')
df_arope_ccaa_limpio = cargar_pickle('df_arope_ccaa')
df_gasto_quintil_limpio = cargar_pickle('df_epf_gasto')
df_ipc_sectorial_limpio = cargar_pickle('df_ipc_sectorial')

# DataFrames Eurostat
df_gini_es = cargar_pickle('df_gini_es')
df_gini_ue27 = cargar_pickle('df_gini_ue27')
df_gini_todos = cargar_pickle('df_gini_todos')
df_arop_es = cargar_pickle('df_arop_es')
df_arop_ue27 = cargar_pickle('df_arop_ue27')
df_arop_eu_todos = cargar_pickle('df_arop_eu_todos')
df_s80s20_es = cargar_pickle('df_s80s20_es')
df_s80s20_ue27 = cargar_pickle('df_s80s20_ue27')
df_s80s20_todos = cargar_pickle('df_s80s20_todos')
df_gap_es = cargar_pickle('df_gap_es')
df_gap_ue27 = cargar_pickle('df_gap_ue27')
df_gap_todos = cargar_pickle('df_gap_todos')
df_impacto_redistrib_es = cargar_pickle('df_impacto_redistrib_es')
df_impacto_redistrib_ue27 = cargar_pickle('df_impacto_redistrib_ue27')

print('✅ Todos los DataFrames cargados correctamente')


✅ Todos los DataFrames cargados correctamente


### 📋 Diccionario de Tablas a Cargar (27 tablas total)

In [3]:
dataframes_a_cargar = {
    # Tablas del INE (13 tablas)
    'INE_IPC_Nacional': df_ipc_anual,
    'INE_Umbral_Pobreza_Hogar': df_umbral_limpio,
    'INE_Carencia_Material_Decil': df_carencia_limpio,
    'INE_AROPE_Edad_Sexo': df_arope_edad_sexo_limpio,
    'INE_AROPE_Hogar': df_arope_hogar_limpio,
    'INE_AROPE_Laboral': df_arope_laboral_limpio,
    'INE_AROPE_CCAA': df_arope_ccaa_limpio,
    'INE_Gini_S80S20_CCAA': df_gini_ccaa_limpio,
    'INE_Renta_Media_Decil': df_renta_decil_limpio,
    'INE_Poblacion_Edad_Sexo_Nacionalidad': df_pob_edad_sexo_nacion_limpio,
    'INE_Poblacion_Edad_Sexo_CCAA': df_pob_edad_sexo_ccaa_limpio,
    'INE_Gasto_Medio_Hogar_Quintil': df_gasto_quintil_limpio,
    'INE_IPC_Sectorial_ECOICOP': df_ipc_sectorial_limpio,
    
    # Tablas de Eurostat (14 tablas: España, UE27, Ranking para 4 indicadores + 2 impacto redistributivo)
    'EUROSTAT_Gini_Espana': df_gini_es,
    'EUROSTAT_Gini_UE27': df_gini_ue27,
    'EUROSTAT_Gini_Ranking': df_gini_todos,
    
    'EUROSTAT_AROP_Espana': df_arop_es,
    'EUROSTAT_AROP_UE27': df_arop_ue27,
    'EUROSTAT_AROP_Ranking': df_arop_eu_todos,
    
    'EUROSTAT_S80S20_Espana': df_s80s20_es,
    'EUROSTAT_S80S20_UE27': df_s80s20_ue27,
    'EUROSTAT_S80S20_Ranking': df_s80s20_todos,
    
    'EUROSTAT_Brecha_Pobreza_Espana': df_gap_es,
    'EUROSTAT_Brecha_Pobreza_UE27': df_gap_ue27,
    'EUROSTAT_Brecha_Pobreza_Ranking': df_gap_todos,
    
    'EUROSTAT_Impacto_Redistributivo_Espana': df_impacto_redistrib_es,
    'EUROSTAT_Impacto_Redistributivo_UE27': df_impacto_redistrib_ue27
}

print(f"Preparadas {len(dataframes_a_cargar)} tablas para cargar en SQL Server.")
print("\n📊 Tablas del INE (13 tablas):")
for i, tabla in enumerate([k for k in dataframes_a_cargar.keys() if k.startswith('INE')], 1):
    print(f"   {i}. {tabla}")

print("\n🇪🇺 Tablas de Eurostat (14 tablas - España + UE27 + Ranking):")
for i, tabla in enumerate([k for k in dataframes_a_cargar.keys() if k.startswith('EUROSTAT')], 1):
    print(f"   {i}. {tabla}")

Preparadas 27 tablas para cargar en SQL Server.

📊 Tablas del INE (13 tablas):
   1. INE_IPC_Nacional
   2. INE_Umbral_Pobreza_Hogar
   3. INE_Carencia_Material_Decil
   4. INE_AROPE_Edad_Sexo
   5. INE_AROPE_Hogar
   6. INE_AROPE_Laboral
   7. INE_AROPE_CCAA
   8. INE_Gini_S80S20_CCAA
   9. INE_Renta_Media_Decil
   10. INE_Poblacion_Edad_Sexo_Nacionalidad
   11. INE_Poblacion_Edad_Sexo_CCAA
   12. INE_Gasto_Medio_Hogar_Quintil
   13. INE_IPC_Sectorial_ECOICOP

🇪🇺 Tablas de Eurostat (14 tablas - España + UE27 + Ranking):
   1. EUROSTAT_Gini_Espana
   2. EUROSTAT_Gini_UE27
   3. EUROSTAT_Gini_Ranking
   4. EUROSTAT_AROP_Espana
   5. EUROSTAT_AROP_UE27
   6. EUROSTAT_AROP_Ranking
   7. EUROSTAT_S80S20_Espana
   8. EUROSTAT_S80S20_UE27
   9. EUROSTAT_S80S20_Ranking
   10. EUROSTAT_Brecha_Pobreza_Espana
   11. EUROSTAT_Brecha_Pobreza_UE27
   12. EUROSTAT_Brecha_Pobreza_Ranking
   13. EUROSTAT_Impacto_Redistributivo_Espana
   14. EUROSTAT_Impacto_Redistributivo_UE27


### 🔌 Configuración de Conexión SQL Server

In [4]:
# Configuración de conexión SQL Server
SERVER = 'localhost'
DATABASE = 'desigualdad'
DRIVER = '{ODBC Driver 17 for SQL Server}'

try:
    # Crear la cadena de conexión para Autenticación de Windows
    connection_string = f'DRIVER={DRIVER};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;'
    
    # Crear el "engine" de SQLAlchemy
    quoted_conn_str = urllib.parse.quote_plus(connection_string)
    engine = create_engine(f'mssql+pyodbc:///?odbc_connect={quoted_conn_str}')
    
    # Probar la conexión
    connection = engine.connect()
    print(f"✅ Conexión exitosa a: {SERVER}/{DATABASE}")
    print(f"   Driver: {DRIVER}")
    connection.close()
    
except Exception as e:
    print(f"❌ Error de conexión: {e}")
    raise


✅ Conexión exitosa a: localhost/desigualdad
   Driver: {ODBC Driver 17 for SQL Server}


### 📤 Carga de Tablas a SQL Server

In [5]:
from datetime import datetime

print("="*80)
print("🚀 INICIANDO CARGA A SQL SERVER")
print("="*80)
inicio = datetime.now()

# Contador de éxitos y errores
tablas_cargadas = 0
errores = []

for nombre_tabla, df in dataframes_a_cargar.items():
    try:
        print(f"\n📊 Cargando: {nombre_tabla}")
        print(f"   Registros: {len(df)}")
        print(f"   Columnas: {list(df.columns)}")
        
        # Cargar el DataFrame a SQL Server
        df.to_sql(
            name=nombre_tabla,
            con=engine,
            if_exists='replace',
            index=False,
            schema='dbo'
        )
        
        tablas_cargadas += 1
        print(f"   ✅ Cargada exitosamente")
        
    except Exception as e:
        errores.append((nombre_tabla, str(e)))
        print(f"   ❌ ERROR: {str(e)}")

# Resumen final
fin = datetime.now()
duracion = (fin - inicio).total_seconds()

print("\n" + "="*80)
print("📊 RESUMEN DE CARGA")
print("="*80)
print(f"✅ Tablas cargadas exitosamente: {tablas_cargadas}/{len(dataframes_a_cargar)}")

if errores:
    print(f"\n❌ Errores encontrados: {len(errores)}")
    for tabla, error in errores:
        print(f"   • {tabla}: {error}")
else:
    print("\n🎉 ¡Todas las tablas se cargaron correctamente!")

print(f"\n⏱️  Tiempo total: {duracion:.2f} segundos")
print(f"🕒 Finalizado: {fin.strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

🚀 INICIANDO CARGA A SQL SERVER

📊 Cargando: INE_IPC_Nacional
   Registros: 24
   Columnas: ['Año', 'IPC_Medio_Anual', 'Inflacion_Anual_%']


   ✅ Cargada exitosamente

📊 Cargando: INE_Umbral_Pobreza_Hogar
   Registros: 32
   Columnas: ['Tipo_Hogar', 'Año', 'Umbral_Euros']
   ✅ Cargada exitosamente

📊 Cargando: INE_Carencia_Material_Decil
   Registros: 1683
   Columnas: ['Item', 'Año', 'Valor', 'Decil']


   ✅ Cargada exitosamente

📊 Cargando: INE_AROPE_Edad_Sexo
   Registros: 408
   Columnas: ['Año', 'Sexo', 'Edad', 'Indicador', 'Valor']


   ✅ Cargada exitosamente

📊 Cargando: INE_AROPE_Hogar
   Registros: 308
   Columnas: ['Año', 'Tipo_Hogar', 'Indicador', 'Valor']
   ✅ Cargada exitosamente

📊 Cargando: INE_AROPE_Laboral
   Registros: 152
   Columnas: ['Sexo', 'Situacion_Laboral', 'Territorio', 'Año', 'AROPE']
   ✅ Cargada exitosamente

📊 Cargando: INE_AROPE_CCAA
   Registros: 680
   Columnas: ['Año', 'CCAA', 'Indicador', 'Valor']


   ✅ Cargada exitosamente

📊 Cargando: INE_Gini_S80S20_CCAA
   Registros: 340
   Columnas: ['Territorio', 'Año', 'Gini', 'S80/S20']


   ✅ Cargada exitosamente

📊 Cargando: INE_Renta_Media_Decil
   Registros: 176
   Columnas: ['Decil', 'Año', 'Media', 'Mediana']
   ✅ Cargada exitosamente

📊 Cargando: INE_Poblacion_Edad_Sexo_Nacionalidad
   Registros: 1380
   Columnas: ['Año', 'Sexo', 'Edad', 'Poblacion']


   ✅ Cargada exitosamente

📊 Cargando: INE_Poblacion_Edad_Sexo_CCAA
   Registros: 10260
   Columnas: ['Año', 'CCAA', 'Sexo', 'Edad', 'Poblacion']


   ✅ Cargada exitosamente

📊 Cargando: INE_Gasto_Medio_Hogar_Quintil
   Registros: 5616
   Columnas: ['Año', 'Quintil', 'Grupo_Gasto', 'Tipo_Valor', 'Valor']
   ✅ Cargada exitosamente

📊 Cargando: INE_IPC_Sectorial_ECOICOP
   Registros: 0
   Columnas: ['Año', 'Categoria_ECOICOP', 'IPC_Indice', 'Inflacion_Sectorial_%']
   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_Gini_Espana
   Registros: 10
   Columnas: ['Gini', 'age', 'age_label', 'geo_code', 'geo_name', 'Año']
   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_Gini_UE27
   Registros: 10
   Columnas: ['Gini', 'age', 'age_label', 'geo_code', 'geo_name', 'Año']


   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_Gini_Ranking
   Registros: 365
   Columnas: ['Gini', 'geo_code', 'geo_name', 'Año']
   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_AROP_Espana
   Registros: 10
   Columnas: ['AROP_%', 'sex', 'sex_label', 'age', 'age_label', 'geo_code', 'geo_name', 'Año']
   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_AROP_UE27
   Registros: 10
   Columnas: ['AROP_%', 'sex', 'sex_label', 'age', 'age_label', 'geo_code', 'geo_name', 'Año']
   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_AROP_Ranking
   Registros: 397
   Columnas: ['AROP_%', 'geo_code', 'geo_name', 'Año']
   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_S80S20_Espana
   Registros: 10
   Columnas: ['S80S20_Ratio', 'age', 'age_label', 'sex', 'sex_label', 'geo_code', 'geo_name', 'Año']
   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_S80S20_UE27
   Registros: 10
   Columnas: ['S80S20_Ratio', 'age', 'age_label', 'sex', 'sex_label', 'geo_code', 'geo_name', 'Año']
   ✅ Cargada exitosamente

📊 Carga

   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_Brecha_Pobreza_Espana
   Registros: 10
   Columnas: ['Brecha_Pobreza_%', 'sex', 'sex_label', 'age', 'age_label', 'geo_code', 'geo_name', 'Año']
   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_Brecha_Pobreza_UE27
   Registros: 10
   Columnas: ['Brecha_Pobreza_%', 'sex', 'sex_label', 'age', 'age_label', 'geo_code', 'geo_name', 'Año']
   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_Brecha_Pobreza_Ranking
   Registros: 378
   Columnas: ['Brecha_Pobreza_%', 'geo_code', 'geo_name', 'Año']
   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_Impacto_Redistributivo_Espana
   Registros: 21
   Columnas: ['Gini_Despues', 'age', 'age_label', 'geo_code', 'geo_name', 'Año', 'Gini_Antes_SinPensiones', 'Gini_Antes_ConPensiones']
   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_Impacto_Redistributivo_UE27
   Registros: 10
   Columnas: ['Gini_Despues', 'age', 'age_label', 'geo_code', 'geo_name', 'Año', 'Gini_Antes_SinPensiones', 'Gini_Antes_ConPensiones']
   ✅ C